In [1]:
import pandas as pd
import os
import librosa
import numpy as np
import math
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline
import librosa.display

In [3]:
audio_dataset_path='C:/Users/aleem/Desktop/Currentlyworking_New_extracted audio wav after cleaning then augmentation/sliding_window_1_sec/data/Al-Quraish/'
metadata=pd.read_csv('C:/Users/aleem/Desktop/Currentlyworking_New_extracted audio wav after cleaning then augmentation/sliding_window_1_sec/data/Al-Quraish/metadata_106.csv')
metadata.head(20)

,slice_file_name,fold,classID,class
0,file2_2.wav,1,106,Quraish
1,file2_3.wav,1,106,Quraish
2,file2_4.wav,1,106,Quraish
3,file2_5.wav,1,106,Quraish
4,1.wav,2,106,Quraish
5,2.wav,2,106,Quraish
6,3.wav,2,106,Quraish
7,4.wav,2,106,Quraish
8,a.wav,4,106,Quraish
9,b.wav,4,106,Quraish


Feature Extraction

In [4]:
def mfcc_features_extractor_1sec(file):
    # audio, sample_rate = librosa.load(file_name) 
    # mfccs_features = librosa.feature.mfcc(y=audio, sr=16000, n_mfcc=40)
    # mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
    
    y, sr = librosa.load(file, sr=16000)
    
    y = y[:192000]  #12 Secs for all audios
    zero_padding = np.zeros(192000 - y.shape[0], dtype=np.float32)
    y = np.concatenate([y, zero_padding])   

    # Define window size and hop length
    win_length = int(sr * 1) # 1 second window
    hop_length = int(win_length / 2) # 50% overlap

    # Extract MFCC features for each window
    mfccs = []
    for i in range(0, len(y)-win_length, hop_length):
        mfcc = librosa.feature.mfcc(y[i:i+win_length], sr=sr, n_mfcc=40)
        mfccs.append(mfcc)

    # Combine MFCC features for all windows
    mfccs = np.concatenate(mfccs, axis=1)
    mfccs = mfccs.T

    # Return final MFCC features
    return mfccs

    
    # return mfccs_scaled_features

In [5]:
# mfcc_extracted_features=[]

# for index_num,row in tqdm(metadata.iterrows()):
#     file_name = os.path.join(os.path.abspath(audio_dataset_path),'fold'+str(row["fold"])+'/',str(row["slice_file_name"]))
#     final_class_labels=row["class"]
    
#     # MFCC Features Extraction
#     mfcc_data= mfcc_features_extractor_1sec(file_name)
#     mfcc_extracted_features.append([mfcc_data,final_class_labels])
#     # print(mfcc_extracted_features)

In [6]:
mfcc_data_by_fold = {}

# Iterate through the DataFrame and extract MFCC features
for index_num, row in tqdm(metadata.iterrows()):
    file_name = os.path.join(os.path.abspath(audio_dataset_path), 'fold' + str(row["fold"]) + '/', str(row["slice_file_name"]))
    final_class_labels = row["class"]

    # MFCC Features Extraction
    mfcc_data = mfcc_features_extractor_1sec(file_name)
    
    # Store the MFCC data in the dictionary by fold number
    fold_num = row["fold"]
    if fold_num not in mfcc_data_by_fold:
        mfcc_data_by_fold[fold_num] = {
            "data": [],
            "labels": []
        }
    mfcc_data_by_fold[fold_num]["data"].append(mfcc_data)
    mfcc_data_by_fold[fold_num]["labels"].append(final_class_labels)

# Now, convert the data in mfcc_data_by_fold to NumPy arrays
for fold_num, data_dict in mfcc_data_by_fold.items():
    data_dict["data"] = np.array(data_dict["data"])
    data_dict["labels"] = np.array(data_dict["labels"])

0it [00:00, ?it/s]C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[ 0.02106941  0.01891056  0.01492148 ... -0.06744021 -0.13894397
 -0.1536758 ] as keyword args. From version 0.10 passing these as positional arguments will result in an error
C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[-0.26821512 -0.26087615 -0.21239032 ... -0.07497843 -0.03001095
 -0.01152745] as keyword args. From version 0.10 passing these as positional arguments will result in an error
C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[-0.0019083  -0.0140522  -0.13190421 ...  0.02163285  0.03079749
  0.07775214] as keyword args. From version 0.10 passing these as positional arguments will result in an error
C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[ 0.02861324  0.07

In [7]:
fold_num = 1  # Replace with the fold number you're interested in
data_shape = mfcc_data_by_fold[fold_num]["data"].shape
labels_shape = mfcc_data_by_fold[fold_num]["labels"].shape

print(f"Shape of data for fold {fold_num}: {data_shape}")
print(f"Shape of labels for fold {fold_num}: {labels_shape}")

Shape of data for fold 1: (4, 704, 40)
Shape of labels for fold 1: (4,)


In [8]:
mfcc_data_by_fold[3]["data"].shape

(4, 704, 40)

In [9]:
# Iterate through the dictionary and reshape the data arrays for all fold_num values
for fold_num, data_dict in mfcc_data_by_fold.items():
    data_dict["data"] = data_dict["data"].reshape(data_dict["data"].shape + (1,))
    

In [10]:
mfcc_data_by_fold[1]["data"].shape

(4, 704, 40, 1)

After Training Work, Deep Features 

In [11]:
from keras.models import load_model
from keras.models import Model

# Load the saved model
loaded_model = load_model('Model_CNN2D_startwithnumpylengthsamemfcc_1sec_4032files_90_10_size_result 100_96.29.h5')

# Get the name of the intermediate layer from the loaded model summary
intermediate_layer_name = 'dense'

# Find the index of the intermediate layer in the model's layers list
intermediate_layer_index = [i for i, layer in enumerate(loaded_model.layers) if layer.name == intermediate_layer_name][0]

# Define a new model that includes layers up to the intermediate layer
intermediate_layer_model = Model(inputs=loaded_model.input,
                                  outputs=loaded_model.layers[intermediate_layer_index].output)

# List of inputs for each fold's MFCC data
input_data_list = [mfcc_data_by_fold[fold]["data"] for fold in range(1, 4)]  # Adjust the range as needed

# List to store intermediate layer outputs for each fold
intermediate_layer_outputs = []

# Iterate through the input data for each fold and predict the intermediate layer output
for input_data in input_data_list:
    intermediate_output = intermediate_layer_model.predict(input_data)
    intermediate_layer_outputs.append(intermediate_output)

# Now, intermediate_layer_outputs contains the intermediate layer outputs for each fold's input data


1/1 [==============================] - 0s 62ms/step


In [12]:
intermediate_layer_outputs = np.array(intermediate_layer_outputs)

In [13]:
intermediate_layer_outputs.shape

(3, 4, 1024)

In [14]:
intermediate_layer_outputs

array([[[-0.7276035 ,  0.99981445,  0.998796  , ...,  0.95670485,
         -0.9805024 , -0.90138733],
        [-0.8397517 , -0.9939267 ,  0.9582128 , ...,  0.99904543,
         -1.        , -0.99666953],
        [-0.81174606, -0.97635436,  0.9998601 , ...,  0.9788843 ,
         -0.9968211 , -0.9996808 ],
        [ 0.58770555,  1.        , -0.64959055, ...,  0.9998399 ,
         -1.        , -0.9999405 ]],

       [[ 0.557053  ,  0.99998736,  0.9909279 , ...,  0.9632142 ,
         -0.994463  , -0.93231887],
        [-0.9101296 ,  0.9999868 ,  0.99987453, ...,  0.9972992 ,
         -0.99999964, -0.997324  ],
        [-0.9878285 , -0.95298856,  0.9982212 , ...,  0.9988926 ,
         -0.9975187 , -0.9998727 ],
        [-0.97835565,  0.9551669 ,  0.73783225, ...,  0.9999931 ,
         -1.        , -0.9999929 ]],

       [[-0.8892034 , -0.99995756,  0.98722696, ...,  0.9282733 ,
         -0.99248487, -0.99046725],
        [-0.999308  , -1.        ,  0.99999684, ...,  0.9692326 ,
         -0.

In [15]:
mean_across_folds = np.mean(intermediate_layer_outputs, axis=0)

In [17]:
al_quraish_mean = mean_across_folds

In [18]:
al_quraish_mean.shape

(4, 1024)

In [19]:
al_quraish_mean

array([[-0.3532513 ,  0.33328143,  0.99231696, ...,  0.9493974 ,
        -0.9891501 , -0.94139117],
       [-0.91639644, -0.33131328,  0.986028  , ...,  0.9885257 ,
        -0.9999425 , -0.99765474],
       [-0.9326136 , -0.97644764,  0.9992969 , ...,  0.99023587,
        -0.9941419 , -0.9986746 ],
       [-0.46355   ,  0.31838894,  0.36274722, ...,  0.9999444 ,
        -0.9999991 , -0.9999543 ]], dtype=float32)

In [20]:
# Specify the file path where you want to save the .npy file
file_path = 'al_quraish_mean.npy'

# Save the array as a .npy file
np.save(file_path, al_quraish_mean)

Testing of Single Audio

In [21]:
file_name = 'C:/Users/aleem/Desktop/Currentlyworking_New_extracted audio wav after cleaning then augmentation/sliding_window_1_sec/data/quraish_testing.wav'


mfcc_extracted_features_single=[]

mfcc_extracted_features_single= mfcc_features_extractor_1sec(file_name)

mfcc_extracted_features_single = np.array(mfcc_extracted_features_single)

mfcc_extracted_features_single = mfcc_extracted_features_single.reshape((1, 704, 40, 1))



C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[-0.00690354 -0.00708194  0.00814009 ... -0.02031602 -0.03851673
 -0.03577803] as keyword args. From version 0.10 passing these as positional arguments will result in an error
C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[0.04146133 0.02658811 0.03397983 ... 0.04617729 0.10076655 0.21527734] as keyword args. From version 0.10 passing these as positional arguments will result in an error
C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[-0.03517909 -0.04793629 -0.05519976 ...  0.03259142  0.01832691
 -0.00214196] as keyword args. From version 0.10 passing these as positional arguments will result in an error
C:\Users\aleem\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: FutureWarning: Pass y=[ 0.2300222   0.15509711  0.09771885 ... -0

In [22]:
mfcc_extracted_features_single

array([[[[ -266.35306  ],
         [   33.303066 ],
         [  -40.37836  ],
         ...,
         [   -2.9622352],
         [    4.2220993],
         [    9.093099 ]],

        [[ -242.79277  ],
         [   21.98935  ],
         [  -44.403572 ],
         ...,
         [    4.3935604],
         [    9.239249 ],
         [   19.231083 ]],

        [[ -249.2378   ],
         [   11.613199 ],
         [  -49.201927 ],
         ...,
         [   10.621393 ],
         [   10.409626 ],
         [   22.362904 ]],

        ...,

        [[-1131.3708   ],
         [    0.       ],
         [    0.       ],
         ...,
         [    0.       ],
         [    0.       ],
         [    0.       ]],

        [[-1131.3708   ],
         [    0.       ],
         [    0.       ],
         ...,
         [    0.       ],
         [    0.       ],
         [    0.       ]],

        [[-1131.3708   ],
         [    0.       ],
         [    0.       ],
         ...,
         [    0.       ],
        

In [23]:
from keras.models import load_model
from keras.models import Model

# Load the saved model
loaded_model = load_model('Model_CNN2D_startwithnumpylengthsamemfcc_1sec_4032files_90_10_size_result 100_96.29.h5')

# Get the name of the intermediate layer from the loaded model summary
intermediate_layer_name = 'dense'

# Find the index of the intermediate layer in the model's layers list
intermediate_layer_index = [i for i, layer in enumerate(loaded_model.layers) if layer.name == intermediate_layer_name][0]

# Define a new model that includes layers up to the intermediate layer
intermediate_layer_model = Model(inputs=loaded_model.input,
                                  outputs=loaded_model.layers[intermediate_layer_index].output)

# Assuming you already have mfcc_extracted_features of shape (704, 40, 1)
# Replace this with your actual input data

# Predict the intermediate layer output for the input data
intermediate_output = intermediate_layer_model.predict(mfcc_extracted_features_single)

# Now, intermediate_output contains the intermediate layer output for mfcc_extracted_features


1/1 [==============================] - 0s 134ms/step


In [24]:
intermediate_output

array([[-0.9878285 , -0.95298845,  0.9982212 , ...,  0.9988925 ,
        -0.9975187 , -0.99987257]], dtype=float32)

In [25]:
intermediate_output.shape

(1, 1024)

In [26]:
intermediate_layer_outputs

array([[[-0.7276035 ,  0.99981445,  0.998796  , ...,  0.95670485,
         -0.9805024 , -0.90138733],
        [-0.8397517 , -0.9939267 ,  0.9582128 , ...,  0.99904543,
         -1.        , -0.99666953],
        [-0.81174606, -0.97635436,  0.9998601 , ...,  0.9788843 ,
         -0.9968211 , -0.9996808 ],
        [ 0.58770555,  1.        , -0.64959055, ...,  0.9998399 ,
         -1.        , -0.9999405 ]],

       [[ 0.557053  ,  0.99998736,  0.9909279 , ...,  0.9632142 ,
         -0.994463  , -0.93231887],
        [-0.9101296 ,  0.9999868 ,  0.99987453, ...,  0.9972992 ,
         -0.99999964, -0.997324  ],
        [-0.9878285 , -0.95298856,  0.9982212 , ...,  0.9988926 ,
         -0.9975187 , -0.9998727 ],
        [-0.97835565,  0.9551669 ,  0.73783225, ...,  0.9999931 ,
         -1.        , -0.9999929 ]],

       [[-0.8892034 , -0.99995756,  0.98722696, ...,  0.9282733 ,
         -0.99248487, -0.99046725],
        [-0.999308  , -1.        ,  0.99999684, ...,  0.9692326 ,
         -0.

Euclidian Distance with Mean

In [28]:
import numpy as np

# Assuming you have al_fil_mean with shape (5, 1024) and intermediate_output with shape (1, 1024)
# Calculate the Euclidean distances between intermediate_output and each row in al_fil_mean
distances = np.linalg.norm(al_quraish_mean - intermediate_output, axis=1)

# Find the index of the closest row in al_fil_mean
closest_index = np.argmin(distances)

# The value at closest_index in al_fil_mean is the closest to intermediate_output
closest_value = al_quraish_mean[closest_index]

# Print the closest_index and closest_value
print(f"Index of closest value in al_quraish_mean: {closest_index+1}")
print(f"Closest value in al_quraish_mean: {closest_value}")

Index of closest value in al_fil_mean: 3
Closest value in al_fil_mean: [-0.9326136  -0.97644764  0.9992969  ...  0.99023587 -0.9941419
 -0.9986746 ]


Euclidian Distance Without Mean

In [26]:
# # Reshape intermediate_output to match the shape of intermediate_layer_outputs
# intermediate_output_reshaped = intermediate_output.reshape((1, 1, 1024))

# # Initialize variables to keep track of the closest indices and distances
# closest_indices = None
# min_distances = None
# closest_set = None  # Initialize variable to track the closest set

# for i, intermediate_set in enumerate(intermediate_layer_outputs):
#     # Calculate the Euclidean distances between intermediate_output and intermediate_set
#     distances = np.linalg.norm(intermediate_set - intermediate_output_reshaped, axis=2)
    
#     # Find the index of the minimum distance
#     current_closest_index = np.argmin(distances)
    
#     # Update the closest index, distances, and closest set if this set is closer
#     if closest_indices is None or np.mean(distances) < np.mean(min_distances):
#         closest_indices = current_closest_index
#         min_distances = distances
#         closest_set = i

# # Print the closest index in intermediate_layer_outputs and which set it belongs to
# print(f"Closest index in intermediate_layer_outputs: {closest_indices}")
# if closest_set == 0:
#     print("It is closest to intermediate_layer_outputs[0]")
# elif closest_set == 1:
#     print("It is closest to intermediate_layer_outputs[1]")
# elif closest_set == 2:
#     print("It is closest to intermediate_layer_outputs[2]")